In [427]:
import pandas as pd
import numpy as np
import datetime as dt

In [428]:
eventsData = pd.read_csv("data/Eventos(+completo).csv",low_memory = False)

In [429]:
eventsData.columns

Index(['Unnamed: 0', 'timestamp', 'event', 'person', 'url', 'sku', 'model',
       'condition', 'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'Parte del dia'],
      dtype='object')

In [430]:
eventsData = eventsData.drop("Unnamed: 0", axis=1)

In [431]:
geoData = eventsData[["person","event","city","region","country"]]

In [432]:
deviceUsersData = eventsData[["person","browser_version","operating_system_version","screen_resolution","device_type"]]

In [433]:
productData = eventsData[["person","event","sku","model","condition","storage","skus"]]

In [434]:
infoUserData = eventsData[["person","timestamp","search_term","staticpage","campaign_source","search_engine","channel","new_vs_returning"]]

In [435]:
timeData = eventsData[["person","timestamp","Parte del dia","event"]]

In [436]:
personColor = eventsData[["person","event","color"]]

Busco crear nuevo features 

temporales

In [437]:
timeData.head()

,person,timestamp,Parte del dia,event
0,4886f805,2018-05-18 00:11:59,Noche,viewed product
1,ad93850f,2018-05-18 00:11:27,Noche,viewed product
2,0297fc1e,2018-05-18 00:11:16,Noche,viewed product
3,2d681dd8,2018-05-18 00:11:14,Noche,viewed product
4,cccea85e,2018-05-18 00:11:09,Noche,viewed product


In [438]:
users_partedeldia = timeData[["person","Parte del dia"]]

In [439]:
users_partedeldia = pd.get_dummies(users_partedeldia, columns=["Parte del dia"])

In [440]:
users_partedeldia = users_partedeldia.groupby("person",as_index = False).sum()

In [441]:
#Funcion que pasa a formato datetime las fechas que recibimos.
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

timeData["timestamp"]= datetime(timeData["timestamp"],"%Y-%m-%dT%H:%M:%S")

type(timeData["timestamp"][0])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


pandas.tslib.Timestamp

In [442]:
timeData["year"] = timeData["timestamp"].map(lambda x: x.year)
timeData["month"] = timeData["timestamp"].map(lambda x: x.month)
timeData["day"] = timeData["timestamp"].map(lambda x: x.day)
timeData["hour"] = timeData["timestamp"].map(lambda x: x.hour)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

In [443]:
last_15days = timeData[["person","event","month","day"]]

In [444]:

last_15days.head()

,person,event,month,day
0,4886f805,viewed product,5,18
1,ad93850f,viewed product,5,18
2,0297fc1e,viewed product,5,18
3,2d681dd8,viewed product,5,18
4,cccea85e,viewed product,5,18


In [445]:
last_15days = last_15days[last_15days["month"] == 5]

In [446]:
last_15days = last_15days[last_15days["day"] > 14]
last_15days = last_15days[["person","event"]]
last_15days.head()

,person,event
0,4886f805,viewed product
1,ad93850f,viewed product
2,0297fc1e,viewed product
3,2d681dd8,viewed product
4,cccea85e,viewed product


In [447]:
last_15days["count_activity"] = 1

In [448]:
last_15days = pd.get_dummies(last_15days, columns=["event"])
last_15days.head()

,person,count_activity,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
0,4886f805,1,0,0,0,0,0,0,0,0,0,1,0
1,ad93850f,1,0,0,0,0,0,0,0,0,0,1,0
2,0297fc1e,1,0,0,0,0,0,0,0,0,0,1,0
3,2d681dd8,1,0,0,0,0,0,0,0,0,0,1,0
4,cccea85e,1,0,0,0,0,0,0,0,0,0,1,0


In [449]:
last_15days = last_15days.groupby("person",as_index=False).sum()
last_15days = last_15days[["person","count_activity","event_conversion","event_checkout","event_viewed product"]]
last_15days.head()

,person,count_activity,event_conversion,event_checkout,event_viewed product
0,0008ed71,6,0.0,3.0,0.0
1,00091926,283,0.0,2.0,250.0
2,000ba417,206,1.0,6.0,153.0
3,000c79fe,17,0.0,1.0,3.0
4,000e4d9e,411,0.0,1.0,339.0


In [450]:
last_15days.columns = ["person","count_activity","conversiones_15lastdays","checkout_15lastdays","viewed_15lastday"]
last_15days.head()

,person,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday
0,0008ed71,6,0.0,3.0,0.0
1,00091926,283,0.0,2.0,250.0
2,000ba417,206,1.0,6.0,153.0
3,000c79fe,17,0.0,1.0,3.0
4,000e4d9e,411,0.0,1.0,339.0


In [451]:
timeData.head()

,person,timestamp,Parte del dia,event,year,month,day,hour
0,4886f805,2018-05-18 00:11:59,Noche,viewed product,2018,5,18,0
1,ad93850f,2018-05-18 00:11:27,Noche,viewed product,2018,5,18,0
2,0297fc1e,2018-05-18 00:11:16,Noche,viewed product,2018,5,18,0
3,2d681dd8,2018-05-18 00:11:14,Noche,viewed product,2018,5,18,0
4,cccea85e,2018-05-18 00:11:09,Noche,viewed product,2018,5,18,0


In [452]:
timeData["minute"] = timeData["timestamp"].map(lambda x: x.minute)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [453]:
timeData.head()

,person,timestamp,Parte del dia,event,year,month,day,hour,minute
0,4886f805,2018-05-18 00:11:59,Noche,viewed product,2018,5,18,0,11
1,ad93850f,2018-05-18 00:11:27,Noche,viewed product,2018,5,18,0,11
2,0297fc1e,2018-05-18 00:11:16,Noche,viewed product,2018,5,18,0,11
3,2d681dd8,2018-05-18 00:11:14,Noche,viewed product,2018,5,18,0,11
4,cccea85e,2018-05-18 00:11:09,Noche,viewed product,2018,5,18,0,11


In [454]:
timeData["year"].value_counts()

2018    2341681
Name: year, dtype: int64

In [455]:
timeData = timeData.drop("year",axis = 1)

In [456]:
timeData = pd.get_dummies(timeData, columns=["event"])
timeData.head()

,person,timestamp,Parte del dia,month,day,hour,minute,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
0,4886f805,2018-05-18 00:11:59,Noche,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
1,ad93850f,2018-05-18 00:11:27,Noche,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
2,0297fc1e,2018-05-18 00:11:16,Noche,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
3,2d681dd8,2018-05-18 00:11:14,Noche,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
4,cccea85e,2018-05-18 00:11:09,Noche,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0


In [457]:
timeData = timeData.drop(["timestamp","Parte del dia"], axis=1)

In [458]:
timeData.head()

,person,month,day,hour,minute,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
0,4886f805,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
1,ad93850f,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
2,0297fc1e,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
3,2d681dd8,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0
4,cccea85e,5,18,0,11,0,0,0,0,0,0,0,0,0,1,0


In [459]:
timeData = timeData.drop("minute",axis=1)

In [460]:
secionesUsuarios = timeData.groupby(["person","month","day","hour"]).sum()

In [461]:
secionesUsuarios["cant_seciones"] = 1

In [462]:
secionesUsuarios.head()

event_ad campaign hit  event_brand listing  \
person   month day hour                                               
0008ed71 5     17  12                      0.0                  0.0   
                   13                      0.0                  0.0   
                   16                      0.0                  0.0   
00091926 5     3   22                      0.0                  0.0   
               5   3                       0.0                  0.0   

                         event_checkout  event_conversion  \
person   month day hour                                     
0008ed71 5     17  12               1.0               0.0   
                   13               1.0               0.0   
                   16               1.0               0.0   
00091926 5     3   22               0.0               0.0   
               5   3                0.0               0.0   

                         event_generic listing  event_lead  \
person   month day hour                                      
0008ed71 5     17  12                      0.0         0.0   
                   13                      0.0         0.0   
                   16                      1.0         0.0   
00091926 5     3   22                      0.0         0.0   
               5   3                       0.0         0.0   

                         event_search engine hit  event_searched products  \
person   month day hour                                                     
0008ed71 5     17  12                        0.0                      0.0   
                   13                        0.0                      0.0   
                   16                        0.0                      0.0   
00091926 5     3   22                        0.0                      0.0   
               5   3                         0.0                      0.0   

                         event_staticpage  event_viewed product  \
person   month day hour                                           
0008ed71 5     17  12                 0.0                   0.0   
                   13                 0.0                   0.0   
                   16                 0.0                   0.0   
00091926 5     3   22                 0.0                   6.0   
               5   3                  0.0                   5.0   

                         event_visited site  cant_seciones  
person   month day hour                                     
0008ed71 5     17  12                   0.0              1  
                   13                   1.0              1  
                   16                   1.0              1  
00091926 5     3   22                   1.0              1  
               5   3                    1.0              1

In [463]:
secionesUsuarios = secionesUsuarios[["cant_seciones","event_checkout","event_conversion","event_ad campaign hit","event_lead"]]
secionesUsuarios.head()

cant_seciones  event_checkout  event_conversion  \
person   month day hour                                                    
0008ed71 5     17  12                1             1.0               0.0   
                   13                1             1.0               0.0   
                   16                1             1.0               0.0   
00091926 5     3   22                1             0.0               0.0   
               5   3                 1             0.0               0.0   

                         event_ad campaign hit  event_lead  
person   month day hour                                     
0008ed71 5     17  12                      0.0         0.0  
                   13                      0.0         0.0  
                   16                      0.0         0.0  
00091926 5     3   22                      0.0         0.0  
               5   3                       0.0         0.0

In [464]:
secionesUsuarios = secionesUsuarios.reset_index()

In [465]:
secionesUsuarios.head()

,person,month,day,hour,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead
0,0008ed71,5,17,12,1,1.0,0.0,0.0,0.0
1,0008ed71,5,17,13,1,1.0,0.0,0.0,0.0
2,0008ed71,5,17,16,1,1.0,0.0,0.0,0.0
3,00091926,5,3,22,1,0.0,0.0,0.0,0.0
4,00091926,5,5,3,1,0.0,0.0,0.0,0.0


In [466]:
secionesUsuarios = secionesUsuarios.drop(["day","hour","month"],axis=1)

In [467]:
secionesUsuarios = secionesUsuarios.groupby(["person"]).sum().fillna(0)

In [468]:
secionesUsuarios.head()

,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead
person,,,,,
0008ed71,3,3.0,0.0,0.0,0.0
00091926,37,2.0,0.0,15.0,0.0
00091a7a,1,0.0,0.0,1.0,0.0
000ba417,9,6.0,1.0,1.0,0.0
000c79fe,1,1.0,0.0,1.0,0.0


In [469]:
secionesUsuarios = secionesUsuarios.reset_index()

In [470]:
secionesUsuarios.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead'],
      dtype='object')

In [471]:
secionesUsuarios.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead
0,0008ed71,3,3.0,0.0,0.0,0.0
1,00091926,37,2.0,0.0,15.0,0.0
2,00091a7a,1,0.0,0.0,1.0,0.0
3,000ba417,9,6.0,1.0,1.0,0.0
4,000c79fe,1,1.0,0.0,1.0,0.0


Por dispositivo que usa el usuario para entrar al sitio

In [472]:
deviceUsersData.head()

,person,browser_version,operating_system_version,screen_resolution,device_type
0,4886f805,Chrome Mobile 64.0,Android 7,360x640,Smartphone
1,ad93850f,Chrome Mobile 66.0,Android 5.1.1,360x640,Smartphone
2,0297fc1e,Chrome Mobile 61.0,Android 6.0.1,360x640,Smartphone
3,2d681dd8,Chrome 66.0,Windows 8.1,1360x768,Computer
4,cccea85e,Chrome 66.0,Windows 7,1536x864,Computer


In [473]:
deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0]).value_counts()

Windows       1262240
Android        975740
iOS             73279
Mac             10206
Linux            9456
Ubuntu           3313
nan              2870
Chrome           2376
Other            1048
Tizen             531
BlackBerry        518
Fedora             92
FreeBSD             8
Symbian             4
Name: operating_system_version, dtype: int64

In [474]:
deviceUsersData["operating_system_version"] = deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [475]:
deviceUsersData["device_type"].value_counts()

Computer      1276520
Smartphone    1028684
Tablet          29896
Unknown          3711
Name: device_type, dtype: int64

In [476]:
deviceUsersData["device_type"] = deviceUsersData["device_type"].map(lambda x: "otros" if(x == "Tablet" or x=="Unknown") else x)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [477]:
device_user_type = deviceUsersData[["person","device_type"]]

In [478]:
device_user_type["count"] = 1

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [479]:
device_user_type = device_user_type.groupby(["person","device_type"]).max().unstack().fillna(0).reset_index()

In [480]:
device_user_type.head()

person    count                 
device_type           Computer Smartphone otros
0            0008ed71      1.0        0.0   0.0
1            00091926      1.0        0.0   0.0
2            00091a7a      0.0        1.0   0.0
3            000ba417      1.0        0.0   0.0
4            000c79fe      0.0        1.0   0.0

In [481]:
multiindex = device_user_type.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_type.columns = ind
device_user_type.head()

,person,Computer,Smartphone,otros
0,0008ed71,1.0,0.0,0.0
1,00091926,1.0,0.0,0.0
2,00091a7a,0.0,1.0,0.0
3,000ba417,1.0,0.0,0.0
4,000c79fe,0.0,1.0,0.0


In [482]:
device_user_system = deviceUsersData[["person","operating_system_version"]]

In [483]:
device_user_system["count"] = 1

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [484]:
device_user_system = device_user_system.groupby(["person","operating_system_version"]).max().unstack().fillna(0).reset_index()

In [485]:
device_user_system.head()

person   count                                   \
operating_system_version           Android BlackBerry Chrome Fedora FreeBSD   
0                         0008ed71     0.0        0.0    0.0    0.0     0.0   
1                         00091926     0.0        0.0    0.0    0.0     0.0   
2                         00091a7a     1.0        0.0    0.0    0.0     0.0   
3                         000ba417     0.0        0.0    0.0    0.0     0.0   
4                         000c79fe     1.0        0.0    0.0    0.0     0.0   

                                                                             \
operating_system_version Linux  Mac Other Symbian Tizen Ubuntu Windows  iOS   
0                          0.0  0.0   0.0     0.0   0.0    0.0     1.0  0.0   
1                          0.0  0.0   0.0     0.0   0.0    0.0     1.0  0.0   
2                          0.0  0.0   0.0     0.0   0.0    0.0     0.0  0.0   
3                          0.0  0.0   0.0     0.0   0.0    0.0     1.0  0.0   
4                          0.0  0.0   0.0     0.0   0.0    0.0     0.0  0.0   

                               
operating_system_version  nan  
0                         0.0  
1                         0.0  
2                         0.0  
3                         0.0  
4                         0.0

In [486]:
multiindex = device_user_system.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_system.columns = ind
device_user_system.head()

,person,Android,BlackBerry,Chrome,Fedora,FreeBSD,Linux,Mac,Other,Symbian,Tizen,Ubuntu,Windows,iOS,nan
0,0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,00091a7a,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,000c79fe,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [487]:
device_user_system = device_user_system[["person","Windows","Android","iOS"]]

Informacion usuarios

In [488]:
infoUserData.head()

,person,timestamp,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning
0,4886f805,2018-05-18 00:11:59,NaN,NaN,NaN,NaN,NaN,NaN
1,ad93850f,2018-05-18 00:11:27,NaN,NaN,NaN,NaN,NaN,NaN
2,0297fc1e,2018-05-18 00:11:16,NaN,NaN,NaN,NaN,NaN,NaN
3,2d681dd8,2018-05-18 00:11:14,NaN,NaN,NaN,NaN,NaN,NaN
4,cccea85e,2018-05-18 00:11:09,NaN,NaN,NaN,NaN,NaN,NaN


In [489]:
dic_terminos = {}
for busqueda in infoUserData["search_term"]:
    for term in str(busqueda).split(" "):
        if (not term in dic_terminos.keys()):
            dic_terminos[term] = 1
        else:
            dic_terminos[term]+=1
            

In [490]:
list_terminos = sorted(dic_terminos.items(), key=lambda kv: kv[1])


In [491]:
list_terminos = list_terminos[::-1]

In [492]:
list_terminos.pop(0)

('nan', 2227918)

In [493]:
terminos = []
for name,count in list_terminos:
    if(len(name)>2):
        terminos.append((name,count))
        

In [494]:
len(terminos)

3640

In [495]:
terminos_populares = terminos[:300]

In [496]:
infoUserData["search_term"] = infoUserData["search_term"].fillna("0")

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [497]:
def verificarPopularidadElTermino(string,terminospopulares):
    if (string==0):
        return False
    for palabra in string.split(" "):
        
        if(palabra in [name for name,count in terminospopulares]):
            return True
        
    return False

In [498]:
infoUserData["Busca_terminos_populares"] = infoUserData["search_term"].map(lambda x:1 if(verificarPopularidadElTermino(x,terminos_populares)) else 0)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [499]:
infoUserData["Busca_terminos_populares"].value_counts()

0    2251274
1      90407
Name: Busca_terminos_populares, dtype: int64

In [500]:
infoUserData.head()

,person,timestamp,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,Busca_terminos_populares
0,4886f805,2018-05-18 00:11:59,0,NaN,NaN,NaN,NaN,NaN,0
1,ad93850f,2018-05-18 00:11:27,0,NaN,NaN,NaN,NaN,NaN,0
2,0297fc1e,2018-05-18 00:11:16,0,NaN,NaN,NaN,NaN,NaN,0
3,2d681dd8,2018-05-18 00:11:14,0,NaN,NaN,NaN,NaN,NaN,0
4,cccea85e,2018-05-18 00:11:09,0,NaN,NaN,NaN,NaN,NaN,0


In [501]:
infoUserData["channel"].value_counts()

Paid        91753
Direct      50675
Referral    33990
Organic     25867
Social       1406
Email         371
Unknown         7
Name: channel, dtype: int64

In [502]:
infoUserData["search_engine"].value_counts() #Tira todo a google

Google    105195
Bing         676
Yahoo        519
Ask           16
Name: search_engine, dtype: int64

In [503]:
infoUserData["staticpage"].value_counts()

CustomerService                      5239
FaqEcommerce                         1317
AboutUs                              1217
Quiosks                               868
trust-trocafone                       632
TermsAndConditionsReturnEcommerce     528
galaxy-s8                             380
Conditions                            285
how-to-sell                           234
how-to-buy                            232
TermsAndConditionsEcommerce           119
club-trocafone                        107
PrivacyEcommerce                       28
black_friday                           15
Name: staticpage, dtype: int64

In [504]:
infoUserData["campaign_source"].value_counts()

google              123354
criteo               24931
rtbhouse             23540
zanox                 7013
emblue                3653
afilio                1913
manifest              1365
FacebookAds           1301
voxus                  971
rakuten                843
bing                   839
datacrush              433
indexa                 313
buscape                271
Facebook               203
afiliado                92
blog                    88
mercadopago             74
yotpo                   54
onsite                  12
FacebookSocial          12
MARKETING SOCIAL         7
gizmodo                  4
Name: campaign_source, dtype: int64

In [505]:
person_campaing = infoUserData[["person","campaign_source"]]
person_channel = infoUserData[["person","channel"]]

In [506]:
person_term = infoUserData[["person","Busca_terminos_populares"]]

In [507]:
person_term.head()

,person,Busca_terminos_populares
0,4886f805,0
1,ad93850f,0
2,0297fc1e,0
3,2d681dd8,0
4,cccea85e,0


In [508]:
person_term = person_term.groupby("person",as_index = False).sum()

In [509]:
person_term.head()

,person,Busca_terminos_populares
0,0008ed71,0
1,00091926,0
2,00091a7a,0
3,000ba417,0
4,000c79fe,8


In [510]:
person_campaing = person_campaing.dropna()

In [511]:
person_campaing.head()

,person,campaign_source
9,45baf068,google
14,45baf068,google
20,29ebb414,google
34,2a724d87,google
38,15ea8012,google


In [512]:
person_campaing = pd.get_dummies(person_campaing, columns=["campaign_source"])

In [513]:
person_campaing.head()

,person,campaign_source_Facebook,campaign_source_FacebookAds,campaign_source_FacebookSocial,campaign_source_MARKETING SOCIAL,campaign_source_afiliado,campaign_source_afilio,campaign_source_bing,campaign_source_blog,campaign_source_buscape,...,campaign_source_google,campaign_source_indexa,campaign_source_manifest,campaign_source_mercadopago,campaign_source_onsite,campaign_source_rakuten,campaign_source_rtbhouse,campaign_source_voxus,campaign_source_yotpo,campaign_source_zanox
9,45baf068,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14,45baf068,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20,29ebb414,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
34,2a724d87,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
38,15ea8012,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [514]:
person_campaing = person_campaing[["person","campaign_source_google","campaign_source_criteo","campaign_source_rtbhouse"]]

In [515]:
person_campaing = person_campaing.groupby("person",as_index=False).sum()

In [516]:
person_channel = person_channel.dropna()

In [517]:
person_channel.head()

,person,channel
2136629,4640420b,Paid
2136630,4640420b,Paid
2136631,4640420b,Paid
2136632,4640420b,Paid
2136633,4640420b,Organic


In [518]:
person_channel = pd.get_dummies(person_channel, columns=["channel"])

In [519]:
person_channel.head()

,person,channel_Direct,channel_Email,channel_Organic,channel_Paid,channel_Referral,channel_Social,channel_Unknown
2136629,4640420b,0,0,0,1,0,0,0
2136630,4640420b,0,0,0,1,0,0,0
2136631,4640420b,0,0,0,1,0,0,0
2136632,4640420b,0,0,0,1,0,0,0
2136633,4640420b,0,0,1,0,0,0,0


In [520]:
person_channel = person_channel[["person","channel_Direct","channel_Paid","channel_Organic","channel_Referral"]]

In [521]:
person_channel = person_channel.groupby("person",as_index=False).sum()

Productos visto por el usuario

In [522]:
productData.head()

,person,event,sku,model,condition,storage,skus
0,4886f805,viewed product,9288.0,samsung galaxy j7 prime,Excelente,32GB,NaN
1,ad93850f,viewed product,304.0,iphone 5s,Muito Bom,32GB,NaN
2,0297fc1e,viewed product,6888.0,iphone 6s,Muito Bom,64GB,NaN
3,2d681dd8,viewed product,11890.0,iphone 7,Bom,128GB,NaN
4,cccea85e,viewed product,7517.0,lg g4 h818p,Excelente,32GB,NaN


In [523]:
productData["condition"].value_counts()

Bom                   547617
Excelente             378697
Muito Bom             359053
Bom - Sem Touch ID     31742
Novo                    3421
Name: condition, dtype: int64

In [524]:
productData["storage"].value_counts()

16GB     442096
32GB     430283
64GB     228413
128GB     99345
8GB       96019
256GB     17623
4GB        5409
512MB      1342
Name: storage, dtype: int64

In [525]:
productData["model"] = productData["model"].fillna("Desconocido")
productData["marca"] = productData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [526]:
productData["marca"].value_counts()

Desconocido    893560
apple          643135
samsung        564336
motorola       177813
lg              27472
sony            16672
lenovo          11848
asus             5041
quantum          1737
ofertas            26
list               24
nokia              15
outros              1
xiaomi              1
Name: marca, dtype: int64

In [527]:
productData["version"] = productData["model"].map(lambda x:" ".join(x.split(" ")[0:2]) if(x.split(" ")[0] in ["iphone","ipad"]) else " ".join(x.split(" ")[1:3]))
productData["version"].value_counts()

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


                 905497
iphone 6         147600
iphone 6s        144700
iphone 5s        108058
iphone 7         107571
galaxy s7         96659
galaxy s6         76769
galaxy s8         58797
galaxy j7         51695
galaxy j5         48966
iphone se         39086
moto g4           38921
galaxy a5         36331
galaxy s5         34274
galaxy note       33937
iphone 5c         29797
galaxy a7         28334
moto x            27335
moto g5           25950
iphone 5          19876
moto g3           19328
moto z            18165
iphone 4s         15478
galaxy gran       13724
moto g2           13386
moto x2           13155
galaxy j2         10800
galaxy s4          9093
iphone             8926
xperia z3          8548
                  ...  
muv pro              46
gear s3              45
muv up               38
x screen             34
razr i               31
moto g1a             29
live                 28
galaxy young         27
pre venda            26
l80 dual             21
ipad            

In [528]:
users_view_list = productData[["person","skus"]]

In [529]:
users_view_list["skus"].isnull().value_counts()

True     1835732
False     505949
Name: skus, dtype: int64

In [530]:
productData = productData[(productData["event"] == "checkout")]

In [531]:
productData.head()

,person,event,sku,model,condition,storage,skus,marca,version
33,15ea8012,checkout,12802.0,samsung galaxy s8 plus,Excelente,64GB,NaN,samsung,galaxy s8
60,43790d8f,checkout,10378.0,motorola moto g4 plus,Bom,32GB,NaN,motorola,moto g4
76,d614c608,checkout,6343.0,samsung galaxy j5,Bom,16GB,NaN,samsung,galaxy j5
133,55d1e0ee,checkout,6663.0,samsung galaxy core plus duos tv,Excelente,4GB,NaN,samsung,galaxy core
147,bb78c182,checkout,7631.0,motorola moto g3 4g,Bom,16GB,NaN,motorola,moto g3


In [532]:
productData = productData[["person","marca","condition","version","storage"]]

In [533]:
productData.head()

,person,marca,condition,version,storage
33,15ea8012,samsung,Excelente,galaxy s8,64GB
60,43790d8f,motorola,Bom,moto g4,32GB
76,d614c608,samsung,Bom,galaxy j5,16GB
133,55d1e0ee,samsung,Excelente,galaxy core,4GB
147,bb78c182,motorola,Bom,moto g3,16GB


In [534]:
productData["marca"].value_counts()

samsung     27838
apple       26771
motorola     8442
lg           1140
lenovo        573
sony          416
asus          102
quantum        33
Name: marca, dtype: int64

In [535]:
productData["version"].value_counts()

iphone 6          6988
iphone 6s         5658
iphone 5s         5619
galaxy j5         4246
galaxy j7         3649
galaxy s7         3633
iphone 7          3621
galaxy s6         2921
galaxy s8         2350
moto g4           2035
moto g2           1802
galaxy a5         1657
galaxy s5         1363
galaxy gran       1336
iphone 5c         1223
moto g5           1198
iphone se         1118
moto g3            994
galaxy j2          937
galaxy note        877
galaxy a7          839
iphone 4s          816
iphone 5           804
galaxy j1          787
galaxy win         653
moto x             632
moto z             615
galaxy core        607
moto x2            466
iphone 4g          436
                  ... 
ipad air            39
g4 h815p            37
g3 stylus           36
xperia z5           29
k4                  26
k8                  25
g3 beat             21
zenfone 2           20
 x                  19
g4 beat             19
nexus 5             17
g4 stylus           17
go 4g      

In [536]:
productData["condition"].value_counts()

Bom                   28194
Excelente             18543
Muito Bom             16459
Bom - Sem Touch ID     2073
Novo                     46
Name: condition, dtype: int64

In [537]:
productData= pd.get_dummies(productData, columns=["marca","condition","version","storage"])

In [538]:
productData = productData.groupby("person",as_index=False).sum()

In [539]:
productData.columns

Index(['person', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_quantum', 'marca_samsung', 'marca_sony',
       'condition_Bom',
       ...
       'version_zenfone go', 'version_zenfone selfie', 'storage_128GB',
       'storage_16GB', 'storage_256GB', 'storage_32GB', 'storage_4GB',
       'storage_512MB', 'storage_64GB', 'storage_8GB'],
      dtype='object', length=114)

In [540]:
productData = productData.drop(["marca_quantum","condition_Novo"],axis = 1)

In [541]:
productData.head()

,person,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_samsung,marca_sony,condition_Bom,condition_Bom - Sem Touch ID,...,version_zenfone go,version_zenfone selfie,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB
0,0008ed71,1,0,0,1,0,1,0,2,0,...,0,0,0,0,0,2,0,0,1,0
1,00091926,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,000ba417,0,0,0,0,1,5,0,4,0,...,0,0,0,6,0,0,0,0,0,0
3,000c79fe,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,000e4d9e,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [542]:
productData.columns

Index(['person', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID',
       ...
       'version_zenfone go', 'version_zenfone selfie', 'storage_128GB',
       'storage_16GB', 'storage_256GB', 'storage_32GB', 'storage_4GB',
       'storage_512MB', 'storage_64GB', 'storage_8GB'],
      dtype='object', length=112)

In [543]:
productData = productData[['person', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom',"version_galaxy j7","version_galaxy j5","version_galaxy s7","version_iphone 6","version_iphone 5s","version_iphone 6s","version_moto g4","version_moto g2","version_galaxy s6",'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB',
       'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB']]

In [544]:
productData.head()

,person,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_samsung,marca_sony,condition_Bom,condition_Bom - Sem Touch ID,...,version_moto g2,version_galaxy s6,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB
0,0008ed71,1,0,0,1,0,1,0,2,0,...,0,1,0,0,0,2,0,0,1,0
1,00091926,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,000ba417,0,0,0,0,1,5,0,4,0,...,0,0,0,6,0,0,0,0,0,0
3,000c79fe,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,000e4d9e,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [545]:
users_view_list.head()

,person,skus
0,4886f805,NaN
1,ad93850f,NaN
2,0297fc1e,NaN
3,2d681dd8,NaN
4,cccea85e,NaN


In [546]:
users_view_list["skus"] = users_view_list["skus"].fillna(0)

In [547]:
users_view_list["skus"] = users_view_list["skus"].map(lambda x:0 if(x == 0) else 1)

In [548]:
users_view_list = users_view_list.groupby("person",as_index=False).max()

In [549]:
new_last15days = infoUserData[["person","timestamp","new_vs_returning"]]

In [550]:
new_last15days.head()

,person,timestamp,new_vs_returning
0,4886f805,2018-05-18 00:11:59,NaN
1,ad93850f,2018-05-18 00:11:27,NaN
2,0297fc1e,2018-05-18 00:11:16,NaN
3,2d681dd8,2018-05-18 00:11:14,NaN
4,cccea85e,2018-05-18 00:11:09,NaN


In [551]:
new_last15days["timestamp"]= datetime(new_last15days["timestamp"],"%Y-%m-%dT%H:%M:%S")
new_last15days["month"] = new_last15days["timestamp"].map(lambda x: x.month)
new_last15days["day"] = new_last15days["timestamp"].map(lambda x: x.day)
new_last15days.head()


/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

,person,timestamp,new_vs_returning,month,day
0,4886f805,2018-05-18 00:11:59,NaN,5,18
1,ad93850f,2018-05-18 00:11:27,NaN,5,18
2,0297fc1e,2018-05-18 00:11:16,NaN,5,18
3,2d681dd8,2018-05-18 00:11:14,NaN,5,18
4,cccea85e,2018-05-18 00:11:09,NaN,5,18


In [552]:
new_last15days["new_vs_returning"].value_counts()

Returning    165827
New           38242
Name: new_vs_returning, dtype: int64

In [553]:
new_last15days = new_last15days[new_last15days["month"] == 5]
new_last15days = new_last15days[new_last15days["day"] >14]
new_last15days.head()

,person,timestamp,new_vs_returning,month,day
0,4886f805,2018-05-18 00:11:59,NaN,5,18
1,ad93850f,2018-05-18 00:11:27,NaN,5,18
2,0297fc1e,2018-05-18 00:11:16,NaN,5,18
3,2d681dd8,2018-05-18 00:11:14,NaN,5,18
4,cccea85e,2018-05-18 00:11:09,NaN,5,18


In [554]:
new_last15days["new_15lastdays"] = new_last15days["new_vs_returning"] == "New"

In [555]:
new_last15days["new_15lastdays"].value_counts()

False    1377983
True       25484
Name: new_15lastdays, dtype: int64

In [556]:
new_last15days = new_last15days[["person","new_15lastdays"]]
new_last15days.head()

,person,new_15lastdays
0,4886f805,False
1,ad93850f,False
2,0297fc1e,False
3,2d681dd8,False
4,cccea85e,False


In [557]:
new_last15days = new_last15days.groupby("person",as_index=False).max()

In [558]:
new_last15days["new_15lastdays"] = new_last15days["new_15lastdays"].map(lambda x: 1 if(x) else 0) 

Features geograficos

In [559]:
geoData.head()

,person,event,city,region,country
0,4886f805,viewed product,Cantagalo,Rio de Janeiro,Brazil
1,ad93850f,viewed product,São Paulo,Sao Paulo,Brazil
2,0297fc1e,viewed product,Niterói,Rio de Janeiro,Brazil
3,2d681dd8,viewed product,Juquitiba,Sao Paulo,Brazil
4,cccea85e,viewed product,Osasco,Sao Paulo,Brazil


In [560]:
geoData = geoData[geoData["event"] == "conversion"]

In [561]:
region = geoData[["person","region"]]

In [562]:
region = region.drop_duplicates(subset = "person", keep = "first")

In [563]:
(region["region"].value_counts() > 50).value_counts()

False    29
True     15
Name: region, dtype: int64

In [564]:
region["region"] = region["region"].fillna(0)

In [565]:
sell_region = region["region"].value_counts()

In [566]:
region["ventas_region"] = region["region"].map(lambda x: sell_region[x]) 

In [567]:
region.head()

,person,region,ventas_region
195,49c19e32,Sao Paulo,1213
724,39df97e0,Minas Gerais,421
744,380c0e60,Sao Paulo,1213
1198,35ee0cc8,Unknown,756
2124,c5a0bc36,Unknown,756


In [568]:
esBrasilero = geoData[["person","country"]]

In [569]:
esBrasilero["Brasilero"] = esBrasilero["country"].map(lambda x: 1 if(x == "Brazil") else 0)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [570]:
esBrasilero = esBrasilero[["person","Brasilero"]]

In [571]:
esBrasilero = esBrasilero.groupby("person",as_index=False).max()

Colores

In [572]:
personColor.head()

,person,event,color
0,4886f805,viewed product,Dourado
1,ad93850f,viewed product,Cinza espacial
2,0297fc1e,viewed product,Prateado
3,2d681dd8,viewed product,Vermelho
4,cccea85e,viewed product,Branco


In [573]:
personColor["color"].value_counts().head()

Preto             314925
Dourado           275212
Cinza espacial    151770
Branco            126496
Prateado          114284
Name: color, dtype: int64

In [574]:
personColor = personColor.dropna()

In [575]:
personColor["event"].value_counts()

viewed product    1248124
checkout            65315
conversion           7091
Name: event, dtype: int64

In [576]:
bestcolorseller = personColor[personColor["event"] == "checkout"]
colorsellers = bestcolorseller["color"].value_counts()

In [577]:
colorsellers.head(12)

Preto             17249
Dourado           16295
Cinza espacial     6673
Branco             6601
Prateado           4601
Ouro Rosa          3550
Rosa               1694
Prata              1297
Preto Matte        1135
Platinum            979
Azul                756
Cinza               684
Name: color, dtype: int64

In [578]:
personColor = pd.get_dummies(personColor, columns=["color"])
personColor.head()

,person,event,color_Amarelo,color_Ametista,color_Azul,color_Azul Escuro,color_Azul Safira,color_Azul Topázio,color_Bambu,color_Black Piano,...,color_Rouge,color_Roxo,color_Silver,color_Titânio,color_Turquesa,color_Verde,color_Verde Petroleo,color_Verde Água,color_Vermelho,color_Ônix
0,4886f805,viewed product,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ad93850f,viewed product,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0297fc1e,viewed product,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2d681dd8,viewed product,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,cccea85e,viewed product,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [579]:
personColor = personColor[personColor["event"] == "viewed product"]

In [580]:
personColor = personColor[["person","color_Preto","color_Dourado","color_Branco","color_Cinza espacial"]]
personColor.head()

,person,color_Preto,color_Dourado,color_Branco,color_Cinza espacial
0,4886f805,0,1,0,0
1,ad93850f,0,0,0,1
2,0297fc1e,0,0,0,0
3,2d681dd8,0,0,0,0
4,cccea85e,0,0,1,0


In [581]:
personColor = personColor.groupby("person").sum()

In [582]:
personColor.head()

,color_Preto,color_Dourado,color_Branco,color_Cinza espacial
person,,,,
00091926,79.0,101.0,3.0,65.0
00091a7a,0.0,1.0,0.0,1.0
000ba417,71.0,14.0,15.0,1.0
000c79fe,0.0,0.0,0.0,0.0
000e4d9e,149.0,46.0,85.0,14.0


Mergeo todos mis resultados de los nuevos features

In [583]:
secionesUsuarios.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead
0,0008ed71,3,3.0,0.0,0.0,0.0
1,00091926,37,2.0,0.0,15.0,0.0
2,00091a7a,1,0.0,0.0,1.0,0.0
3,000ba417,9,6.0,1.0,1.0,0.0
4,000c79fe,1,1.0,0.0,1.0,0.0


In [584]:
device_user_type.head()

,person,Computer,Smartphone,otros
0,0008ed71,1.0,0.0,0.0
1,00091926,1.0,0.0,0.0
2,00091a7a,0.0,1.0,0.0
3,000ba417,1.0,0.0,0.0
4,000c79fe,0.0,1.0,0.0


In [585]:
person_term.head()

,person,Busca_terminos_populares
0,0008ed71,0
1,00091926,0
2,00091a7a,0
3,000ba417,0
4,000c79fe,8


In [586]:
productData.head()

,person,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_samsung,marca_sony,condition_Bom,condition_Bom - Sem Touch ID,...,version_moto g2,version_galaxy s6,storage_128GB,storage_16GB,storage_256GB,storage_32GB,storage_4GB,storage_512MB,storage_64GB,storage_8GB
0,0008ed71,1,0,0,1,0,1,0,2,0,...,0,1,0,0,0,2,0,0,1,0
1,00091926,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,000ba417,0,0,0,0,1,5,0,4,0,...,0,0,0,6,0,0,0,0,0,0
3,000c79fe,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,000e4d9e,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [587]:
users_view_list.head()

,person,skus
0,0008ed71,1
1,00091926,1
2,00091a7a,1
3,000ba417,1
4,000c79fe,1


In [588]:
secionesUsuarios.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead'],
      dtype='object')

In [589]:
secionesUsuariosReducido = secionesUsuarios#[["person","cant_seciones5","cant_seciones4","cant_seciones3","event_checkout5","event_checkout4","event_conversion5","event_conversion4","event_ad campaign hit4","event_ad campaign hit5","event_lead5","event_lead4"]]

In [590]:
secionesUsuariosReducido.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead
0,0008ed71,3,3.0,0.0,0.0,0.0
1,00091926,37,2.0,0.0,15.0,0.0
2,00091a7a,1,0.0,0.0,1.0,0.0
3,000ba417,9,6.0,1.0,1.0,0.0
4,000c79fe,1,1.0,0.0,1.0,0.0


In [591]:
dataTrocafone = pd.merge(secionesUsuariosReducido,users_view_list,on="person",how = "outer")

In [592]:
dataTrocafone.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus
0,0008ed71,3,3.0,0.0,0.0,0.0,1
1,00091926,37,2.0,0.0,15.0,0.0,1
2,00091a7a,1,0.0,0.0,1.0,0.0,1
3,000ba417,9,6.0,1.0,1.0,0.0,1
4,000c79fe,1,1.0,0.0,1.0,0.0,1


In [593]:
dataTrocafone = pd.merge(dataTrocafone,person_term,on="person",how="outer")

In [594]:
dataTrocafone = pd.merge(dataTrocafone,device_user_type,on="person",how="outer")

In [595]:
dataTrocafone["Computer"] = dataTrocafone["Computer"].fillna(0)

In [596]:
dataTrocafone["Smartphone"] = dataTrocafone["Smartphone"].fillna(0)

In [597]:
dataTrocafone["otros"] = dataTrocafone["otros"].fillna(0)

In [598]:
dataTrocafone = dataTrocafone.fillna(0)

In [599]:
dataTrocafone.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead', 'skus',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros'],
      dtype='object')

In [600]:
dataTrocafone.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,Smartphone,otros
0,0008ed71,3,3.0,0.0,0.0,0.0,1,0,1.0,0.0,0.0
1,00091926,37,2.0,0.0,15.0,0.0,1,0,1.0,0.0,0.0
2,00091a7a,1,0.0,0.0,1.0,0.0,1,0,0.0,1.0,0.0
3,000ba417,9,6.0,1.0,1.0,0.0,1,0,1.0,0.0,0.0
4,000c79fe,1,1.0,0.0,1.0,0.0,1,8,0.0,1.0,0.0


In [601]:
dataTrocafone = pd.merge(dataTrocafone,productData,on="person",how="outer")

In [602]:
dataTrocafone.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead', 'skus',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB'],
      dtype='object')

In [603]:
users_partedeldia.head()

,person,Parte del dia_Madrugada,Parte del dia_Mañana,Parte del dia_Noche,Parte del dia_Tarde
0,0008ed71,0.0,0.0,1.0,5.0
1,00091926,196.0,0.0,252.0,0.0
2,00091a7a,0.0,0.0,0.0,10.0
3,000ba417,0.0,57.0,43.0,106.0
4,000c79fe,0.0,0.0,17.0,0.0


In [604]:
dataTrocafone = pd.merge(dataTrocafone,users_partedeldia,on="person",how="left")

In [605]:
dataTrocafone = pd.merge(dataTrocafone,person_campaing,on="person",how="outer")

In [606]:
dataTrocafone = pd.merge(dataTrocafone,person_channel,on="person",how="outer")

In [607]:
dataTrocafone = pd.merge(dataTrocafone,esBrasilero,on="person",how="outer")

In [608]:
dataTrocafone = dataTrocafone.fillna(0)

In [609]:
dataTrocafone = pd.merge(dataTrocafone,region,on="person",how="outer")

In [610]:
dataTrocafone = dataTrocafone.drop("region",axis=1)

In [611]:
dataTrocafone["ventas_region"] = dataTrocafone["ventas_region"].fillna(0)

In [612]:
dataTrocafone.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,Smartphone,...,Parte del dia_Tarde,campaign_source_google,campaign_source_criteo,campaign_source_rtbhouse,channel_Direct,channel_Paid,channel_Organic,channel_Referral,Brasilero,ventas_region
0,0008ed71,3,3.0,0.0,0.0,0.0,1,0,1.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,00091926,37,2.0,0.0,15.0,0.0,1,0,1.0,0.0,...,0.0,13.0,1.0,1.0,25.0,8.0,0.0,1.0,0.0,0.0
2,00091a7a,1,0.0,0.0,1.0,0.0,1,0,0.0,1.0,...,10.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,000ba417,9,6.0,1.0,1.0,0.0,1,0,1.0,0.0,...,106.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1213.0
4,000c79fe,1,1.0,0.0,1.0,0.0,1,8,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [613]:
last_15days.head()

,person,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday
0,0008ed71,6,0.0,3.0,0.0
1,00091926,283,0.0,2.0,250.0
2,000ba417,206,1.0,6.0,153.0
3,000c79fe,17,0.0,1.0,3.0
4,000e4d9e,411,0.0,1.0,339.0


In [614]:
dataTrocafone = pd.merge(dataTrocafone,last_15days,how = "left", on="person")

In [615]:
dataTrocafone["count_activity"] = dataTrocafone["count_activity"].fillna(0)

In [616]:
dataTrocafone["conversiones_15lastdays"] = dataTrocafone["conversiones_15lastdays"].fillna(0)
dataTrocafone["checkout_15lastdays"] = dataTrocafone["checkout_15lastdays"].fillna(0)
dataTrocafone["viewed_15lastday"] = dataTrocafone["viewed_15lastday"].fillna(0)

In [617]:
personColor.head()

,color_Preto,color_Dourado,color_Branco,color_Cinza espacial
person,,,,
00091926,79.0,101.0,3.0,65.0
00091a7a,0.0,1.0,0.0,1.0
000ba417,71.0,14.0,15.0,1.0
000c79fe,0.0,0.0,0.0,0.0
000e4d9e,149.0,46.0,85.0,14.0


In [618]:
personColor = personColor.reset_index()

In [619]:
dataTrocafone = pd.merge(dataTrocafone,personColor,how = "left", on="person")

In [620]:
dataTrocafone["color_Preto"] = dataTrocafone["color_Preto"].fillna(0)
dataTrocafone["color_Dourado"] = dataTrocafone["color_Dourado"].fillna(0)
dataTrocafone["color_Branco"] = dataTrocafone["color_Branco"].fillna(0)
dataTrocafone["color_Cinza espacial"] = dataTrocafone["color_Cinza espacial"].fillna(0)

In [621]:
dataTrocafone = pd.merge(dataTrocafone,new_last15days,how = "left", on="person")

In [622]:
dataTrocafone["new_15lastdays"] = dataTrocafone["new_15lastdays"].fillna(0)

In [623]:
dataTrocafone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 61 columns):
person                          38829 non-null object
cant_seciones                   38829 non-null int64
event_checkout                  38829 non-null float64
event_conversion                38829 non-null float64
event_ad campaign hit           38829 non-null float64
event_lead                      38829 non-null float64
skus                            38829 non-null int64
Busca_terminos_populares        38829 non-null int64
Computer                        38829 non-null float64
Smartphone                      38829 non-null float64
otros                           38829 non-null float64
marca_apple                     38829 non-null float64
marca_asus                      38829 non-null float64
marca_lenovo                    38829 non-null float64
marca_lg                        38829 non-null float64
marca_motorola                  38829 non-null float64
marca_samsung     

Armo el set de training y el de test

In [624]:
trainingSet = pd.read_csv("data/labels_training_set.csv", low_memory = False)

In [625]:
trainingSet.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [626]:
trainingSet["person"].count()

19414

In [627]:
dataTrocafone["person"].count()

38829

In [628]:
setTraining = pd.merge(dataTrocafone,trainingSet,on="person",how = "inner")

In [629]:
setTraining.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,Smartphone,...,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,label
0,0008ed71,3,3.0,0.0,0.0,0.0,1,0,1.0,0.0,...,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,000c79fe,1,1.0,0.0,1.0,0.0,1,8,0.0,1.0,...,17.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0
2,001802e4,1,1.0,0.0,5.0,0.0,1,0,0.0,1.0,...,19.0,0.0,1.0,4.0,0.0,2.0,0.0,0.0,1.0,0
3,0019e639,21,15.0,2.0,29.0,0.0,1,3,1.0,0.0,...,202.0,1.0,3.0,89.0,110.0,2.0,24.0,0.0,0.0,0
4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0,0.0,0.0,...,7.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0


In [630]:
setTraining.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead', 'skus',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'channel_Direct',
       'channel_Pa

In [631]:
setTraining["person"].count()

19414

In [632]:
setTraining.to_csv('setEntrenamiento.csv')

In [633]:
kaggle_test = pd.read_csv("data/trocafone_kaggle_test.csv", low_memory = False)

In [634]:
kaggle_test["person"].count()

19415

In [635]:
test = pd.merge(kaggle_test,dataTrocafone,on="person",how="inner")

In [636]:
test["person"].count()

19415

In [637]:
test.to_csv("setTest.csv")